In [1]:
import pandas as pd
from datetime import datetime

In [2]:
df = pd.read_excel('inspectionData_04082020.xlsx')

In [3]:
df.dtypes

bu                                               int64
structureID                                     object
area                                            object
region                                          object
siteName                                        object
structType                                      object
ACQ_Port                                        object
addressLine1                                    object
addressLine2                                    object
city                                            object
state                                           object
postcode                                        object
lit                                             object
lat                                            float64
long                                           float64
buType                                          object
owned_or_managed                                object
inspectMaintainReq                              object
groundElev

In [4]:
df['bu'] = df['bu'].astype(str)
df['lat'] = df['lat'].astype(str)
df['long'] = df['long'].astype(str)

In [5]:
df = df.replace('-','NaN')

In [6]:
df['GBI_Team'] = 'NaN'

In [7]:
special_acq_ports = ['ATT', 'BMI', 'BST', 'DCS']

In [8]:
df.head()

,bu,structureID,area,region,siteName,structType,ACQ_Port,addressLine1,addressLine2,city,...,pre-ModSiteWalk_CM_or_FOT,towerMaintenance,vegetation1,vegetation2,TIA_ClimbedInspection,constructionGC1,constructionGC2,constructionGC3,towerModificationVendor_or_Inspector,GBI_Team
0,805761,A,STA,Texas,TX 143 PLANO,GUYED,ABN,670 Fulgham Rd.,NaN,Plano,...,NaN,2020-02-24 00:00:00,2020-04-09 00:00:00,2020-09-25 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
1,805762,A,STA,Texas,TX 144 FORNEY,GUYED,ABN,10655 FM 740,NaN,Forney,...,NaN,2020-04-04 00:00:00,2020-04-04 00:00:00,2020-10-13 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN
2,805763,A,STA,Texas,TX 145 SHERMAN,GUYED,ABN,Dripping Springs Road,NaN,Denison,...,NaN,2020-10-31 00:00:00,2020-04-06 00:00:00,2020-09-21 00:00:00,NaN,2020-11-30 00:00:00,NaN,NaN,NaN,NaN
3,805764,A,STA,Texas,TX 146 MCKINNEY,GUYED,ABN,5259 Alma,NaN,McKinney,...,2020-12-01 00:00:00,2020-08-07 00:00:00,2020-04-19 00:00:00,2020-10-30 00:00:00,NaN,2020-04-11 00:00:00,2020-11-02 00:00:00,2020-10-30 00:00:00,2020-12-31 00:00:00,NaN
4,805765,A,STA,Texas,TX 147 DENTON,GUYED,ABN,903 South Mayhill Road,NaN,Denton,...,NaN,2020-08-23 00:00:00,2020-04-27 00:00:00,2020-09-07 00:00:00,NaN,2020-06-14 00:00:00,2020-07-04 00:00:00,NaN,NaN,NaN


In [9]:
def assignTeam_specialACQ_Ports(x):
    Team1 = [df['pre-ConstructionSiteWalk1_CM'][x], df['pre-ConstructionSiteWalk2_CM'][x],df['pre-ConstructionSiteWalk3_CM'][x],
                      df['pre-ModSiteWalk_CM_or_FOT'][x]]
    Team2 = [df['towerMaintenance'][x], df['TIA_ClimbedInspection'][x], df['towerModificationVendor_or_Inspector'][x]]
    try:
        if min(d for d in Team1 if isinstance(d, datetime)) <= df['scheduleGBI_Date'][x]:
            if (df['pre-ConstructionSiteWalk1_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'CM1'
            elif (df['pre-ConstructionSiteWalk2_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'CM2'
            elif (df['pre-ConstructionSiteWalk3_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'CM3'
            elif (df['pre-ModSiteWalk_CM_or_FOT'][x] == min(d for d in Team1 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'CM/FOT'
    except:
        try:
            if min(d for d in Team2 if isinstance(d, datetime)) <= df['scheduleGBI_Date'][x]:
                if (df['towerMaintenance'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                    df['GBI_Team'][x] = 'TM'
                elif (df['TIA_ClimbedInspection'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                    df['GBI_Team'][x] = 'TIA'
                elif (df['towerModificationVendor_or_Inspector'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                    df['GBI_Team'][x] = 'TMV'
            else:
                df['GBI_Team'][x] = 'FOT'
        except:
            df['GBI_Team'][x] = 'FOT'

In [10]:
def assignTeam(x):
    Team1 = [df['pre-ConstructionSiteWalk1_CM'][x], df['pre-ConstructionSiteWalk2_CM'][x],df['pre-ConstructionSiteWalk3_CM'][x],
                      df['pre-ModSiteWalk_CM_or_FOT'][x]]
    Team2 = [df['towerMaintenance'][x], df['TIA_ClimbedInspection'][x], df['towerModificationVendor_or_Inspector'][x]]
    try:
        if (df['pre-ConstructionSiteWalk1_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'CM1'
        elif (df['pre-ConstructionSiteWalk2_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'CM2'
        elif (df['pre-ConstructionSiteWalk3_CM'][x] == min(d for d in Team1 if isinstance(d, datetime))):
            df['GBI_Team'][x] = 'CM3'
    except:
        try:
            if (df['pre-ModSiteWalk_CM_or_FOT'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'CM/FOT'
            elif (df['towerMaintenance'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TM'
            elif (df['TIA_ClimbedInspection'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TIA'
            elif (df['towerModificationVendor_or_Inspector'][x] == min(d for d in Team2 if isinstance(d, datetime))):
                df['GBI_Team'][x] = 'TMV'
            else:
                df['GBI_Team'][x] = 'FOT'
        except:
            df['GBI_Team'][x] = 'FOT'

In [ ]:
for x in range(len(df)):
    if df['inspectMaintainReq'][x] in ['Y', 'P']:
        if df['ACQ_Port'][x] in special_acq_ports:
            assignTeam_specialACQ_Ports(x)
        else:
            assignTeam(x) 

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning

In [12]:
df.to_excel('siteVisitAnalysis_Scenario3_output.xlsx')

In [13]:
df['GBI_Team']

0         TM
1         TM
2        CM1
3        CM1
4        CM1
        ... 
40746    CM3
40747     TM
40748    CM2
40749    TIA
40750    CM1
Name: GBI_Team, Length: 40751, dtype: object

In [14]:
visits = df.groupby('GBI_Team').count()

In [15]:
visits

,bu,structureID,area,region,siteName,structType,ACQ_Port,addressLine1,addressLine2,city,...,pre-ConstructionSiteWalk3_CM,pre-ModSiteWalk_CM_or_FOT,towerMaintenance,vegetation1,vegetation2,TIA_ClimbedInspection,constructionGC1,constructionGC2,constructionGC3,towerModificationVendor_or_Inspector
GBI_Team,,,,,,,,,,,,,,,,,,,,,
CM/FOT,147,147,147,147,147,147,147,147,0,147,...,147,147,147,147,147,147,147,147,147,147
CM1,12455,12455,12455,12455,12455,12455,11672,12452,259,12455,...,12455,12455,12455,12455,12455,12455,12455,12455,12455,12455
CM2,2561,2561,2561,2561,2561,2561,2357,2561,98,2561,...,2561,2561,2561,2561,2561,2561,2561,2561,2561,2561
CM3,326,326,326,326,326,326,298,326,11,326,...,326,326,326,326,326,326,326,326,326,326
FOT,13307,13307,13307,13307,13307,13307,12815,13303,163,13306,...,13307,13307,13307,13307,13307,13307,13307,13307,13307,13307
NaN,5165,5165,5165,5165,5165,5165,4978,5139,46,5165,...,5165,5165,5165,5165,5165,5165,5165,5165,5165,5165
TIA,2097,2097,2097,2097,2097,2097,1970,2095,38,2097,...,2097,2097,2097,2097,2097,2097,2097,2097,2097,2097
TM,4652,4652,4652,4652,4652,4652,4115,4651,86,4652,...,4652,4652,4652,4652,4652,4652,4652,4652,4652,4652
TMV,41,41,41,41,41,41,41,41,0,41,...,41,41,41,41,41,41,41,41,41,41


In [16]:
total_FOT_visits = visits['bu'][4]

In [17]:
quantity_of_sites_covered_by_vendors = sum(visits['bu'][6:])

In [18]:
total_FOT_visits

13307

In [19]:
quantity_of_sites_covered_by_vendors

6790